In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import xgcm

In [3]:
import seaduck.eulerian_budget as se

In [4]:
import seaduck as sd

In [5]:
ds = xr.open_zarr('/sciserver/filedb02-02/ocean/wenrui_temp/dic_exp/bgc_tutorial.zarr')

In [6]:
ds.nbytes/1e9

53.69088516

In [7]:
# ds = ds.isel(time = slice(3),time_outer = slice(4))

In [8]:
ds['DICPFLX']

<xarray.DataArray 'DICPFLX' (time: 3000, Z: 15, Y: 64, X: 128)>
dask.array<open_dataset-DICPFLX, shape=(3000, 15, 64, 128), dtype=float32, chunksize=(20, 15, 64, 128), chunktype=numpy.ndarray>
Coordinates:
  * X        (X) float64 1.406 4.219 7.031 9.844 ... 350.2 353.0 355.8 358.6
  * Y        (Y) float64 -88.59 -85.78 -82.97 -80.16 ... 80.16 82.97 85.78 88.59
  * Z        (Z) float64 -25.0 -85.0 -170.0 ... -3.575e+03 -4.19e+03 -4.855e+03
  * time     (time) float64 2.239e+11 2.239e+11 2.24e+11 ... 2.252e+11 2.252e+11
Attributes:
    description:  User-Defined Model-Level Diagnostic  #1
    units:        user-defined

In [9]:
ds.nbytes/1e9

53.69088516

In [10]:
xgcmgrd = se.create_periodic_grid(ds)

In [11]:
tub = sd.OceData(ds)

In [12]:
c_center = np.array(ds['TRAC01'])
p_center = np.array(ds['TRAC03'])
c_center[c_center==0] = np.nan
p_center[p_center==0] = np.nan

In [13]:
c_impl = np.array(ds['DIC'][1:])
p_impl = np.array(ds['PO4'][1:])

In [14]:
ds['utrans'] = ds['UVELMASS']*ds['drF']*ds['dyG']
ds['vtrans'] = ds['VVELMASS']*ds['drF']*ds['dxG']
ds['wtrans'] = ds['WVELMASS']*ds['rA']

In [15]:
rax_buffer = se.buffer_x_periodic(np.array(ds['rA']), 0,1)
ray_buffer = se.buffer_y_periodic(np.array(ds['rA']), 0,1)

In [16]:
u_cfl = np.nan_to_num(np.array(ds['UVELMASS']*ds['dyG']/rax_buffer))
v_cfl = np.nan_to_num(np.array(ds['VVELMASS']*ds['dxG']/ray_buffer))
w_cfl = np.nan_to_num(np.array(ds['WVELMASS']).transpose(1,0,2,3).T/np.array(ds['drF'])).T.transpose(1,0,2,3)

In [17]:
out = xr.Dataset()

In [18]:
out['cwallx'] = xr.DataArray(se.second_order_flux_limiter_x(c_center,u_cfl),dims = ('time','Z','Y','Xp1'))
out['cwally'] = xr.DataArray(se.second_order_flux_limiter_y(c_center,v_cfl),dims = ('time','Z','Yp1','X'))
out['cwallz'] = xr.DataArray(se.second_order_flux_limiter_z_withoutface(c_impl,w_cfl),dims = ('time','Zl','Y','X'))
# out['cwallz'] = xr.DataArray(se.second_order_flux_limiter_z_withoutface(c_center,w_cfl),dims = ('time','Zl','Y','X'))


KeyboardInterrupt



In [ ]:
tp.ind_tend()

In [ ]:
right_ind = tuple([neo.iz-1]+ list(tp.ind_tend_vec((neo.iy, neo. ix), np.ones(len(neo.nprof))*3)))

In [ ]:
out['pwallx'] = xr.DataArray(se.second_order_flux_limiter_x(p_center,u_cfl),dims = ('time','Z','Y','Xp1'))
out['pwally'] = xr.DataArray(se.second_order_flux_limiter_y(p_center,v_cfl),dims = ('time','Z','Yp1','X'))
out['pwallz'] = xr.DataArray(se.second_order_flux_limiter_z_withoutface(p_center,w_cfl),dims = ('time','Zl','Y','X'))

In [ ]:
tub['DFrATr01'] = tub['DFrITr01']+tub['DFrETr01']
out["c_adv"]   = xr.DataArray(se.total_div(tub, xgcmgrd, 'ADVxTr01', 'ADVyTr01', 'ADVrTr01'),dims = ('time','Z','Y','X'))
out["c_dif_h"] = xr.DataArray(se.hor_div(tub, xgcmgrd, 'DFxETr01', 'DFyETr01'),dims = ('time','Z','Y','X'))
out["c_dif_v"] = xr.DataArray(se.ver_div(tub, xgcmgrd, 'DFrATr01'),dims = ('time','Z','Y','X'))

In [ ]:
out['c_adv'][:,0] -= (ds.WTRAC01.isel(Z = 0))/ds.drF[0]

In [ ]:
out['forc_c'] = ds.ForcTr01

In [ ]:
tub['DFrATr03'] = tub['DFrITr03']+tub['DFrETr03']
out["p_adv"]   = xr.DataArray(se.total_div(tub, xgcmgrd, 'ADVxTr03', 'ADVyTr03', 'ADVrTr03'),dims = ('time','Z','Y','X'))
out["p_dif_h"] = xr.DataArray(se.hor_div(tub, xgcmgrd, 'DFxETr03', 'DFyETr03'),dims = ('time','Z','Y','X'))
out["p_dif_v"] = xr.DataArray(se.ver_div(tub, xgcmgrd, 'DFrATr03'),dims = ('time','Z','Y','X'))

In [ ]:
out['p_adv'][:,0] -= (ds.WTRAC03.isel(Z = 0))/ds.drF[0]

In [ ]:
out['forc_p'] = ds.ForcTr03

In [ ]:
it = 0
iz = 0
plt.pcolormesh((ds.Tp_gTr01-out['c_dif_h']-out['c_dif_v']-out['c_adv']-out['forc_c'])[it,iz])
plt.colorbar()

In [ ]:
it = 0
iz = 0
plt.pcolormesh((ds.Tp_gTr03-out['p_dif_h']-out['p_dif_v']-out['p_adv']-out['forc_p'])[it,iz])
plt.colorbar()

In [ ]:
it = 0
iz = 10
plt.pcolormesh((ds.Tp_gTr01-out['c_dif_h']-out['c_dif_v']-out['c_adv']-out['forc_c'])[it,iz])
plt.colorbar()

In [ ]:
it = 0
iz = 10
plt.pcolormesh((ds.Tp_gTr03-out['p_dif_h']-out['p_dif_v']-out['p_adv']-out['forc_p'])[it,iz])
plt.colorbar()

In [ ]:
Rcp = 117
dt = 5*86400

In [ ]:
out['tend_c'] = xr.DataArray(ds.DIC.diff('time_outer').data/dt, dims = ('time','Z','Y','X'))
out['tend_p'] = xr.DataArray(ds.PO4.diff('time_outer').data/dt, dims = ('time','Z','Y','X'))

In [ ]:
out['s_c'] = out['tend_c'] - ds.Tp_gTr01
out['s_p'] = out['tend_p'] - ds.Tp_gTr03

In [ ]:
out['diccarb'] = ds.DICCARB
out['dictflx'] = xr.zeros_like(out['diccarb'])
out['dictflx'][:,0] = ds.DICTFLX

In [ ]:
iz = 1
it = 2
vmax = 1e-12
plt.pcolormesh(out['tend_c'][it,iz]-ds.Tp_gTr01[it,iz]-Rcp*out['s_p'][it,iz]-out['diccarb'][it,iz] - out['dictflx'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
plt.colorbar()

In [ ]:
iz = 1
it = 2
vmax = 1e-25
plt.pcolormesh(out['tend_p'][it,iz]-ds.Tp_gTr03[it,iz]-out['s_p'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
plt.colorbar()

In [ ]:
out['dicpflx'] = ds['DICPFLX']
out['dicbioa'] = ds['DICBIOA']
out['dicrdop'] = ds['DICRDOP']#out['s_p'] - out['dicpflx'] +out['dicbioa']

In [ ]:
iz = 10
it = 2
vmax = 1e-15
plt.pcolormesh(out['s_p'][it,iz]+out['dicbioa'][it,iz] - out['dicpflx'][it,iz] - out['dicrdop'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
plt.colorbar()

In [ ]:
# iz = 0
# it = 2
# vmax = 1e-10
# plt.pcolormesh(out['dicrdop'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
# plt.colorbar()

In [ ]:
# iz = 1
# it = 2
# vmax = 10
# plt.pcolormesh(ds['DICBIOA'][it,iz]/out['dicrdop'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
# plt.colorbar()

In [ ]:
out['res_x_c'] = out['cwallx']*ds['utrans']
out['res_y_c'] = out['cwally']*ds['vtrans']
out['res_z_c'] = out['cwallz']*ds['wtrans']

In [ ]:
out["c_res_adv"] = xr.DataArray(se.total_div(tub, xgcmgrd, 'res_x_c', 'res_y_c', 'res_z_c'),dims = ('time','Z','Y','X'))

In [ ]:
out['res_x_p'] = out['pwallx']*ds['utrans']
out['res_y_p'] = out['pwally']*ds['vtrans']
out['res_z_p'] = out['pwallz']*ds['wtrans']

In [ ]:
out["p_res_adv"] = xr.DataArray(se.total_div(tub, xgcmgrd, 'res_x_p', 'res_y_p', 'res_z_p'),dims = ('time','Z','Y','X'))

In [ ]:
out["e_ua_c"] = out['c_adv']-out['c_res_adv']
out["e_ua_p"] = out['p_adv']-out['p_res_adv']

In [ ]:
iz = 0
it = 1
vmax = 5e-9
plt.pcolormesh(out['c_res_adv'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
plt.colorbar()

In [ ]:
# iz = 1
# it = 1
# vmax = 5e-9
# plt.pcolormesh(out['c_adv'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
# plt.colorbar()

In [ ]:
# iz = 0
# it = 1
# vmax = 5e-9
# plt.pcolormesh(out['dictflx'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
# plt.colorbar()

In [ ]:
# iz = 0
# it = 2
# vmax = 5e-11
# plt.pcolormesh(out['dicrdop'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
# plt.colorbar()

In [ ]:
iz = 0
it = 1
vmax = 5e-12
plt.pcolormesh(out['e_ua_p'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
plt.colorbar()

In [ ]:
iz = 0
it = 1
vmax = 5e-10
plt.pcolormesh(out['e_ua_c'][it,iz],vmax = vmax,vmin = -vmax,cmap = 'bwr')
plt.colorbar()

In [ ]:
out = out.chunk({'time':20,"Z": 15,"Y": 64,"X": 128,"Zl": 15,"Xp1": 129,"Yp1": 65,})

In [ ]:
out.to_zarr('/sciserver/filedb02-02/ocean/wenrui_temp/dic_exp/bgc_tutorial_budget.zarr',mode = 'w')

In [ ]:
thing = np.array(out['s_p'][0])
thing[thing==0] = np.nan
sec = np.nanmean(thing,axis = -1)

vmax = 1e-12
plt.pcolormesh(ds.Y,ds.Z,sec,vmax = vmax, vmin = -vmax, cmap = 'bwr')
plt.xlim(-90,90)
plt.ylim(-4000,0)
plt.colorbar()

In [ ]:
thing = np.array(out['dicpflx'][0])
thing[thing==0] = np.nan
sec = np.nanmean(thing,axis = -1)

vmax = 1e-12
plt.pcolormesh(ds.Y,ds.Z,sec,vmax = vmax, vmin = -vmax, cmap = 'bwr')
plt.xlim(-90,90)
plt.ylim(-4000,0)
plt.colorbar()

In [ ]:
thing = np.array(out['dicrdop'][0])
thing[thing==0] = np.nan
sec = np.nanmean(thing,axis = -1)

vmax = 1e-12
plt.pcolormesh(ds.Y,ds.Z,sec,vmax = vmax, vmin = -vmax, cmap = 'bwr')
plt.xlim(-90,90)
plt.ylim(-4000,0)
plt.colorbar()

In [ ]:
thing = np.array(-out['dicbioa'][0])
thing[thing==0] = np.nan
sec = np.nanmean(thing,axis = -1)

vmax = 1e-10
plt.pcolormesh(ds.Y,ds.Z,sec,vmax = vmax, vmin = -vmax, cmap = 'bwr')
plt.xlim(-90,90)
plt.ylim(-4000,0)
plt.colorbar()

In [ ]:
# plt.pcolormesh(dis_v[1,1])
# plt.colorbar()

In [ ]:
# it = 0
# iz = 4
# plt.pcolormesh((ds.Tp_gTr03-p_dif_h-p_dif_v-p_adv-ds.ForcTr03)[it,iz])
# plt.colorbar()

In [ ]:
# vol = ds.rA*ds.drF*(ds.HFacC).fillna(0)

# adv_h = xgcmgrd.diff(ds['ADVxTr01'],axis = 'X')+ xgcmgrd.diff(ds['ADVyTr01'],axis = 'Y')

# adv_v = xgcmgrd.diff(ds['ADVrTr01'],axis = 'Z',boundary = 'fill',fill_value = '0')

# adv = -adv_h+adv_v

# dif = -xgcmgrd.diff(ds['DFxETr01'],axis = 'X')-xgcmgrd.diff(ds['DFyETr01'],axis = 'Y')+xgcmgrd.diff(ds['DFrETr01']+ds['DFrITr01'],axis = 'Z',boundary = 'fill',fill_value = '0')

# vmax = 1e-13
# plt.pcolormesh((ds.Tp_gTr01-dif/vol-adv/vol-ds.ForcTr01)[2,3],vmax = vmax,vmin = -vmax,cmap = 'bwr')
# plt.colorbar()

In [ ]:
# adv_h = xgcmgrd.diff(ds['ADVxTr01'],axis = 'X')+ xgcmgrd.diff(ds['ADVyTr01'],axis = 'Y')

# adv_v = xgcmgrd.diff(ds['ADVrTr01'],axis = 'Z',boundary = 'fill',fill_value = '0')

# adv = -adv_h+adv_v

# c_adv_h = se.hor_div(tub, xgcmgrd,'ADVxTr01', 'ADVyTr01')

# c_adv_v = se.ver_div(tub, xgcmgrd,'ADVrTr01')

# dis_h = c_adv_h + adv_h/tub['Vol']

# dis_v = c_adv_v - adv_v/tub['Vol']



# dis = (adv/tub['Vol']-out['c_adv'])

# plt.pcolormesh(dis[2,10])
# plt.colorbar()

# plt.pcolormesh(dis_h[1,1])
# plt.colorbar()

In [ ]:
# mean_surf.values

# mean_vel = (ds.WVELMASS.isel(Zl = 0)*ds.rA).sum(dim = ('X','Y')) / (ds.rA.sum())

# mean_vel.values

# plt.pcolormesh(ds.WVELMASS[0,0])
# plt.colorbar()

# plt.pcolormesh(ds.WTRAC01[0,0])
# plt.colorbar()